Copyright **`(c)`** 2023 Andrea Galella s310166

`<andrea.galella@studenti.polito.it>` or `<galella.andrea@gmail.com>`

[`https://github.com/andrea-ga/computational-intelligence`](https://github.com/andrea-ga/computational-intelligence)

References - Course repository:  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence) 

In [47]:
from itertools import product
from functools import reduce
from random import random, randint, shuffle, seed, choice
import numpy as np
from scipy import sparse
from copy import copy
from tabulate import tabulate

In [48]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True

    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [49]:
NUM_POINTS = 100
NUM_POINTS_2 = 1000
NUM_POINTS_3 = 5000

x1 = make_set_covering_problem(NUM_POINTS, NUM_POINTS, .3)
x2 = make_set_covering_problem(NUM_POINTS_2, NUM_POINTS_2, .3)
x3 = make_set_covering_problem(NUM_POINTS_3, NUM_POINTS_3, .3)
x11 = make_set_covering_problem(NUM_POINTS, NUM_POINTS, .7)
x21 = make_set_covering_problem(NUM_POINTS_2, NUM_POINTS_2, .7)
x31 = make_set_covering_problem(NUM_POINTS_3, NUM_POINTS_3, .7)

In [50]:
def evaluate1(state):
    cost = sum(state)
    
    valid = np.all(
        reduce(
            np.logical_or,
            [x1.getrow(i).toarray() for i, t in enumerate(state) if t],
            np.array([False for _ in range(NUM_POINTS)]),
        )
    )
    return valid, -cost if valid else 0

def evaluate2(state):
    cost = sum(state)
    
    valid = np.all(
        reduce(
            np.logical_or,
            [x2.getrow(i).toarray() for i, t in enumerate(state) if t],
            np.array([False for _ in range(NUM_POINTS_2)]),
        )
    )
    return valid, -cost if valid else 0

def evaluate3(state):
    cost = sum(state)
    
    valid = np.all(
        reduce(
            np.logical_or,
            [x3.getrow(i).toarray() for i, t in enumerate(state) if t],
            np.array([False for _ in range(NUM_POINTS_3)]),
        )
    )
    return valid, -cost if valid else 0

def evaluate11(state):
    cost = sum(state)
    
    valid = np.all(
        reduce(
            np.logical_or,
            [x11.getrow(i).toarray() for i, t in enumerate(state) if t],
            np.array([False for _ in range(NUM_POINTS)]),
        )
    )
    return valid, -cost if valid else 0

def evaluate21(state):
    cost = sum(state)
    
    valid = np.all(
        reduce(
            np.logical_or,
            [x21.getrow(i).toarray() for i, t in enumerate(state) if t],
            np.array([False for _ in range(NUM_POINTS_2)]),
        )
    )
    return valid, -cost if valid else 0

def evaluate31(state):
    cost = sum(state)
    
    valid = np.all(
        reduce(
            np.logical_or,
            [x31.getrow(i).toarray() for i, t in enumerate(state) if t],
            np.array([False for _ in range(NUM_POINTS_3)]),
        )
    )
    return valid, -cost if valid else 0

In [51]:
def tweak1(state):
    new_state = copy(state)
    index = randint(0, NUM_POINTS - 1)
    new_state[index] = not new_state[index]
    return new_state

def tweak2(state):
    new_state = copy(state)
    index = randint(0, NUM_POINTS_2 - 1)
    new_state[index] = not new_state[index]
    return new_state

def tweak3(state):
    new_state = copy(state)
    index = randint(0, NUM_POINTS_3 - 1)
    new_state[index] = not new_state[index]
    return new_state

In [52]:
current_state = [choice([True, False]) for _ in range(NUM_POINTS)]
evaluations_num = 0

for step in range(1000):
    new_state = tweak1(current_state)
    if evaluate1(new_state) >= evaluate1(current_state):
        evaluations_num += 2
        current_state = new_state

In [53]:
current_state11 = [choice([True, False]) for _ in range(NUM_POINTS)]
evaluations_num11 = 0

for step in range(1000):
    new_state = tweak1(current_state11)
    if evaluate11(new_state) >= evaluate11(current_state11):
        evaluations_num11 += 2
        current_state11 = new_state

In [54]:
current_state2 = [choice([True, False]) for _ in range(NUM_POINTS_2)]
evaluations_num2 = 0

for step in range(1000):
    new_state = tweak2(current_state2)
    if evaluate2(new_state) >= evaluate2(current_state2):
        evaluations_num2 += 2
        current_state2 = new_state

In [56]:
current_state21 = [choice([True, False]) for _ in range(NUM_POINTS_2)]
evaluations_num21 = 0

for step in range(1000):
    new_state = tweak2(current_state21)
    if evaluate21(new_state) >= evaluate21(current_state21):
        evaluations_num21 += 2
        current_state21 = new_state

In [57]:
current_state3 = [choice([True, False]) for _ in range(NUM_POINTS_3)]
evaluations_num3 = 0

for step in range(1000):
    new_state = tweak3(current_state3)
    if evaluate3(new_state) >= evaluate3(current_state3):
        evaluations_num3 += 2
        current_state3 = new_state

In [58]:
current_state31 = [choice([True, False]) for _ in range(NUM_POINTS_3)]
evaluations_num31 = 0

for step in range(1000):
    new_state = tweak3(current_state31)
    if evaluate31(new_state) >= evaluate31(current_state31):
        evaluations_num31 += 2
        current_state31 = new_state

In [46]:
valid, res = evaluate1(current_state)
valid2, res2 = evaluate2(current_state2)
valid3, res3 = evaluate3(current_state3)
valid11, res11 = evaluate11(current_state11)
valid21, res21 = evaluate21(current_state21)
valid31, res31 = evaluate31(current_state31)

data = [["100", "0.3", -res, valid, evaluations_num],
        ["1000", "0.3", -res2, valid2, evaluations_num2],
        ["5000", "0.3", -res3, valid3, evaluations_num3],
        ["100", "0.7", -res11, valid11, evaluations_num11],
        ["1000", "0.7", -res21, valid21, evaluations_num21],
        ["5000", "0.7", -res31, valid31, evaluations_num31]
       ]

col_names = ["NUM_POINTS", "DENSITY", "COST", "VALIDITY", "NUM_EVALUATIONS"]

print(tabulate(data, headers=col_names, tablefmt="fancy_grid"))

NUM_POINTS | DENSITY | COST       | VALIDITY | NUM_EVALUTATIONS
-----------|---------|------------|----------|-----------------
 100       | 0.3     | -7         | True     | 88
-----------|---------|------------|----------|-----------------
 1000      | 0.3     | -459        | True    | 92
-----------|---------|------------|----------|-----------------
 5000      | 0.3     | -2376        | True    | 96
-----------|---------|------------|----------|-----------------
 100       | 0.7     | -3          | True     | 80
-----------|---------|------------|----------|-----------------
 1000      | 0.7     | -491       | True      | 96
-----------|---------|------------|----------|-----------------
 5000      | 0.7     | -2443       | True      | 102
-----------|---------|------------|----------|-----------------
